In [2]:
import os
import pandas
import numpy
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Average, Input
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from random import randint
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
dataDir = '/home/joel/Documents/KaggleComps/SeedlingId/'
trainDir = os.path.join(dataDir, 'train')
valDir = os.path.join(dataDir, 'validate')
testDir = os.path.join(dataDir, 'test')
sampleSub = pandas.read_csv(os.path.join(dataDir, 'sample_submission.csv'))

In [4]:
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
weeds = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Fat Hen', 'Loose Silky-bent',
         'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill']
crops = ['Common wheat', 'Maize', 'Sugar beet']

In [10]:
# Divide the samples into category folders as expected by flow_from_directory()
minNumSamples = 221
trainVal = []
for cat in os.listdir(os.path.join(dataDir,'trainVal')):
    for file in os.listdir(os.path.join(dataDir,'trainVal', cat)):
        if randint(1,100) < 80:
            os.renames(os.path.join(dataDir,'trainVal',cat,file),os.path.join(dataDir,'train',cat,file))
        else:
            os.renames(os.path.join(dataDir,'trainVal',cat,file),os.path.join(dataDir,'validate',cat,file))

In [5]:
# Stats
T = 0
V = 0
for cat in categories:
    tnum = len(os.listdir(os.path.join(dataDir,'train', cat)))
    vnum = len(os.listdir(os.path.join(dataDir,'validate', cat)))
    print('There are {} pictures of {} in the train folder'.format(tnum, cat))
    print('There are {} pictures of {} in the validate folder'.format(vnum, cat))
    T+=tnum
    V+=vnum
print("There are {} training images and {} validation images".format(T, V))

There are 176 pictures of Black-grass in the train folder
There are 45 pictures of Black-grass in the validate folder
There are 164 pictures of Charlock in the train folder
There are 57 pictures of Charlock in the validate folder
There are 169 pictures of Cleavers in the train folder
There are 52 pictures of Cleavers in the validate folder
There are 175 pictures of Common Chickweed in the train folder
There are 46 pictures of Common Chickweed in the validate folder
There are 182 pictures of Common wheat in the train folder
There are 39 pictures of Common wheat in the validate folder
There are 174 pictures of Fat Hen in the train folder
There are 47 pictures of Fat Hen in the validate folder
There are 179 pictures of Loose Silky-bent in the train folder
There are 42 pictures of Loose Silky-bent in the validate folder
There are 170 pictures of Maize in the train folder
There are 51 pictures of Maize in the validate folder
There are 166 pictures of Scentless Mayweed in the train folder
Th

In [4]:
# Pre-processing on the fly.
trainGenerator=image.ImageDataGenerator(
        rescale=1/255,
        shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True)
testGenerator=image.ImageDataGenerator(rescale=1/255)

In [26]:
# testing results with different input sizes.
sizes=((200,200,3),(250,250,3),(300,300,3),(400,400,3),(500,500,3))
for size in sizes:
    model = Sequential()
    model.add(Conv2D(16, 3, padding='same', input_shape=(size), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(12))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpointer = ModelCheckpoint(filepath='weights/baselineWithAugmentation{}.hdf5'.format(size[0]), verbose=1, save_best_only=True)

    model.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=20), epochs=20,
                       validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2], batch_size=20), callbacks=[checkpointer], verbose=1)

Found 2067 images belonging to 12 classes.
Found 585 images belonging to 12 classes.
Epoch 1/20
104/104 [==============================] - 116s 1s/step - loss: 2.3566 - acc: 0.1720 - val_loss: 2.2013 - val_acc: 0.2393
Epoch 2/20
104/104 [==============================] - 117s 1s/step - loss: 1.7398 - acc: 0.3711 - val_loss: 2.2202 - val_acc: 0.2291
Epoch 3/20
104/104 [==============================] - 121s 1s/step - loss: 1.4465 - acc: 0.4913 - val_loss: 1.1823 - val_acc: 0.6137
Epoch 4/20
104/104 [==============================] - 121s 1s/step - loss: 1.2235 - acc: 0.5954 - val_loss: 1.2985 - val_acc: 0.5812
Epoch 5/20
104/104 [==============================] - 118s 1s/step - loss: 1.1008 - acc: 0.6267 - val_loss: 1.1512 - val_acc: 0.6479
Epoch 6/20
104/104 [==============================] - 119s 1s/step - loss: 0.9749 - acc: 0.6787 - val_loss: 0.9338 - val_acc: 0.7094
Epoch 7/20
104/104 [==============================] - 119s 1s/step - loss: 0.8887 - acc: 0.7227 - val_loss: 0.8492 - 

Epoch 9/20
104/104 [==============================] - 184s 2s/step - loss: 0.7154 - acc: 0.7665 - val_loss: 0.9459 - val_acc: 0.6889
Epoch 10/20
104/104 [==============================] - 180s 2s/step - loss: 0.6738 - acc: 0.7762 - val_loss: 0.7255 - val_acc: 0.7590
Epoch 11/20
104/104 [==============================] - 182s 2s/step - loss: 0.6312 - acc: 0.7972 - val_loss: 0.7061 - val_acc: 0.7607
Epoch 12/20
104/104 [==============================] - 181s 2s/step - loss: 0.5435 - acc: 0.8279 - val_loss: 0.7292 - val_acc: 0.7316
Epoch 13/20
104/104 [==============================] - 181s 2s/step - loss: 0.5497 - acc: 0.8251 - val_loss: 0.8244 - val_acc: 0.7333
Epoch 14/20
104/104 [==============================] - 178s 2s/step - loss: 0.4977 - acc: 0.8303 - val_loss: 0.7307 - val_acc: 0.7573
Epoch 15/20
104/104 [==============================] - 181s 2s/step - loss: 0.4533 - acc: 0.8496 - val_loss: 0.7753 - val_acc: 0.7590
Epoch 16/20
104/104 [==============================] - 177s 2s/

Epoch 18/20
104/104 [==============================] - 254s 2s/step - loss: 0.3533 - acc: 0.8881 - val_loss: 0.6462 - val_acc: 0.7949
Epoch 19/20
104/104 [==============================] - 255s 2s/step - loss: 0.3630 - acc: 0.8889 - val_loss: 0.7691 - val_acc: 0.7863
Epoch 20/20
104/104 [==============================] - 254s 2s/step - loss: 0.3387 - acc: 0.8865 - val_loss: 0.6787 - val_acc: 0.8171
Found 2067 images belonging to 12 classes.
Found 585 images belonging to 12 classes.
Epoch 1/20
104/104 [==============================] - 450s 4s/step - loss: 2.5859 - acc: 0.1947 - val_loss: 1.8600 - val_acc: 0.2872
Epoch 2/20
104/104 [==============================] - 445s 4s/step - loss: 1.6998 - acc: 0.3854 - val_loss: 1.6203 - val_acc: 0.4444
Epoch 3/20
104/104 [==============================] - 444s 4s/step - loss: 1.4326 - acc: 0.4944 - val_loss: 1.2193 - val_acc: 0.5932
Epoch 4/20
104/104 [==============================] - 444s 4s/step - loss: 1.2174 - acc: 0.5906 - val_loss: 1.2351

Epoch 6/20
104/104 [==============================] - 684s 7s/step - loss: 0.9524 - acc: 0.6873 - val_loss: 1.3447 - val_acc: 0.5590
Epoch 7/20
104/104 [==============================] - 682s 7s/step - loss: 0.8576 - acc: 0.7247 - val_loss: 0.7515 - val_acc: 0.7658
Epoch 8/20
104/104 [==============================] - 686s 7s/step - loss: 0.7550 - acc: 0.7692 - val_loss: 0.7674 - val_acc: 0.7761
Epoch 9/20
104/104 [==============================] - 683s 7s/step - loss: 0.6862 - acc: 0.7693 - val_loss: 0.7128 - val_acc: 0.7692
Epoch 10/20
104/104 [==============================] - 685s 7s/step - loss: 0.6311 - acc: 0.7967 - val_loss: 0.9614 - val_acc: 0.7026
Epoch 11/20
104/104 [==============================] - 682s 7s/step - loss: 0.5856 - acc: 0.8178 - val_loss: 0.9434 - val_acc: 0.7145
Epoch 12/20
104/104 [==============================] - 682s 7s/step - loss: 0.5759 - acc: 0.8293 - val_loss: 0.8764 - val_acc: 0.7368
Epoch 13/20
104/104 [==============================] - 679s 7s/ste

In [ ]:
# 250 x 250 has best results
size = (250,250,3)

model = Sequential()
model.add(Conv2D(16, 3, padding='same', input_shape=(size), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(12))
model.add(Activation('softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='weights/baselineWithAugmentation{}.hdf5'.format(size[0]), verbose=1, save_best_only=True)

model.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=20), epochs=20,
                   validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2], batch_size=20), callbacks=[checkpointer], verbose=1)
    

In [ ]:
# Fine tune top layers of InceptionResNetV2, testing different sizes.

sizes = ((299,299,3), (400,400,3), (500,500,3), (600,600,3))

for size in sizes:
    IRmodel = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=size)

    for layer in IRmodel.layers[:618]:
       layer.trainable = False
    for layer in IRmodel.layers[618:]:
       layer.trainable = True

    x=IRmodel.output
    x = MaxPooling2D()(x)
    x = Flatten()(x)
    x = Dense(122, activation='tanh')(x)
    x = Dropout(.5)(x)
    predictions = Dense(12, activation='softmax')(x)

    IRmodel = Model(inputs=IRmodel.input, outputs=predictions)

    IRmodel.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath='weights/IRmodelFT{}.hdf5'.format(size[0]), verbose=1, save_best_only=True)

    IRmodel.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=20), epochs=12,
                          validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2], batch_size=20), callbacks=[checkpointer], verbose=1)
    

219062272/219055592 [==============================] - 39s 0us/step
Found 2067 images belonging to 12 classes.
Found 585 images belonging to 12 classes.
Epoch 1/12
104/104 [==============================] - 1951s 19s/step - loss: 2.6623 - acc: 0.1738 - val_loss: 1.7092 - val_acc: 0.3863
Epoch 2/12
104/104 [==============================] - 1911s 18s/step - loss: 1.8066 - acc: 0.3884 - val_loss: 1.0885 - val_acc: 0.6462
Epoch 3/12
104/104 [==============================] - 1915s 18s/step - loss: 1.2650 - acc: 0.5776 - val_loss: 0.8127 - val_acc: 0.7333
Epoch 4/12
104/104 [==============================] - 1917s 18s/step - loss: 0.9190 - acc: 0.7034 - val_loss: 0.6348 - val_acc: 0.7949
Epoch 5/12
104/104 [==============================] - 1915s 18s/step - loss: 0.7134 - acc: 0.7549 - val_loss: 0.5853 - val_acc: 0.8017
Epoch 6/12
104/104 [==============================] - 1916s 18s/step - loss: 0.6367 - acc: 0.7787 - val_loss: 0.5098 - val_acc: 0.8239
Epoch 7/12
104/104 [=================

In [ ]:
# 299 x 299 seems optimal.

size = (299,299,3)
IRmodel = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=size)

for layer in IRmodel.layers[:618]:
   layer.trainable = False
for layer in IRmodel.layers[618:]:
   layer.trainable = True

x=IRmodel.output
x = GlobalAveragePooling2D()(x)
x = Dense(122, activation='tanh')(x)
x = Dropout(.5)(x)
predictions = Dense(12, activation='softmax')(x)

IRmodel = Model(inputs=IRmodel.input, outputs=predictions)

IRmodel.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='weights/IRmodelGAPFTall{}.hdf5'.format(size[0]), verbose=1, save_best_only=True)

IRmodel.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=20), epochs=20,
                      validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2], batch_size=20),
                      callbacks=[checkpointer], verbose=1)


In [8]:
test=[]
for file in os.listdir(testDir):
    test.append(['test/{}'.format(file), file])
test = pandas.DataFrame(test, columns=["path", "file"])
def maketensor(path):
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    return numpy.expand_dims(x, axis=0)
def maketensors(paths):
    tensorlist = [maketensor(path) for path in tqdm(paths)]
    return numpy.vstack(tensorlist)
testTensors = maketensors(test['path']).astype('float32')/255

100%|██████████| 794/794 [00:10<00:00, 74.70it/s]


In [13]:
IRmodel.load_weights('weights/IRmodelGAPFTall{}.hdf5'.format(size[0]))

seedlingIndexes = [numpy.argmax(IRmodel.predict(numpy.expand_dims(tensor, axis=0))) for tensor in testTensors]
seedlingPredictions=[categories[index] for index in seedlingIndexes]
submission=pandas.DataFrame({'file':test['file'],'species': seedlingPredictions}, columns=["file", "species"])
submission.to_csv('/home/joel/Documents/KaggleComps/SeedlingId/IRmodelGAPFTall{}Submission.csv'.format(size[0]))

***This model acheived 93% accuracy on test set***

In [5]:
"""First attempt at building an ensemble. Trained five seperate models and saved their predictions, then
manually averaged them."""

size = (299,299,3)

for modelnum in range(5):
    batch = 11+modelnum
    IRmodel = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=size)

    for layer in IRmodel.layers[:618]:
       layer.trainable = False
    for layer in IRmodel.layers[618:]:
       layer.trainable = True

    x = IRmodel.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(122, activation='tanh')(x)
    x = Dropout(.5)(x)
    predictions = Dense(12, activation='softmax')(x)

    IRmodel = Model(inputs=IRmodel.input, outputs=predictions)

    IRmodel.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

    checkpointer = ModelCheckpoint(filepath='weights/model{}.hdf5'.format(modelnum), verbose=1, save_best_only=True)

    IRmodel.fit_generator(trainGenerator.flow_from_directory(trainDir, target_size=size[:2], batch_size=batch), epochs=18,
                          validation_data=testGenerator.flow_from_directory(valDir, target_size=size[:2], batch_size=batch),
                          callbacks=[checkpointer], verbose=1)

test=[]
for file in os.listdir(testDir):
    test.append(['test/{}'.format(file), file])
test = pandas.DataFrame(test, columns=["path", "file"])
def maketensor(path):
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    return numpy.expand_dims(x, axis=0)
def maketensors(paths):
    tensorlist = [maketensor(path) for path in tqdm(paths)]
    return numpy.vstack(tensorlist)
testTensors = maketensors(test['path']).astype('float32')/255

predictions = {}

for number in range(5):
    IRmodel.load_weights('weights/model{}.hdf5'.format(number))

    output = []
    for tensor in testTensors:
        output.append(numpy.ndarray.tolist(IRmodel.predict(numpy.expand_dims(tensor, axis=0)))[0])
    seedlingIndexes = [numpy.argmax(image) for image in output]
    seedlingPredictions=[categories[index] for index in seedlingIndexes]
    submission=pandas.DataFrame({'file':test['file'],'species': seedlingPredictions}, columns=["file", "species"])
    submission.to_csv('/home/joel/Documents/KaggleComps/SeedlingId/ensemble/model{}Submission.csv'.format(number))
    predictions[number]=output

pred=pandas.DataFrame.from_dict(predictions, 'index')
invert = pred.to_dict()

indexes=[]

for image in invert:
    frame=pandas.DataFrame(invert[image]).transpose()
    indexes.append(numpy.argmax(frame.mean()))

seedlingPredictions=[categories[index] for index in indexes]
submission=pandas.DataFrame({'file':test['file'],'species': seedlingPredictions}, columns=["file", "species"])
submission.to_csv('/home/joel/Documents/KaggleComps/SeedlingId/ensemble/ensembleSubmission.csv')

Found 2067 images belonging to 12 classes.
Found 585 images belonging to 12 classes.
Epoch 1/18
159/159 [==============================] - 3342s 21s/step - loss: 2.2077 - acc: 0.2521 - val_loss: 1.6564 - val_acc: 0.4650
Epoch 2/18
159/159 [==============================] - 3289s 21s/step - loss: 1.4682 - acc: 0.5506 - val_loss: 1.0916 - val_acc: 0.6906
Epoch 3/18
159/159 [==============================] - 3290s 21s/step - loss: 1.0483 - acc: 0.6812 - val_loss: 0.7916 - val_acc: 0.7778
Epoch 4/18
159/159 [==============================] - 3288s 21s/step - loss: 0.8036 - acc: 0.7484 - val_loss: 0.6338 - val_acc: 0.8085
Epoch 5/18
159/159 [==============================] - 3290s 21s/step - loss: 0.6550 - acc: 0.7900 - val_loss: 0.5440 - val_acc: 0.8376
Epoch 6/18
159/159 [==============================] - 3293s 21s/step - loss: 0.5494 - acc: 0.8273 - val_loss: 0.4812 - val_acc: 0.8530
Epoch 7/18
159/159 [==============================] - 3289s 21s/step - loss: 0.4891 - acc: 0.8321 - val_l

Epoch 10/18
148/148 [==============================] - 3320s 22s/step - loss: 0.3890 - acc: 0.8706 - val_loss: 0.3518 - val_acc: 0.8872
Epoch 11/18
148/148 [==============================] - 3311s 22s/step - loss: 0.3427 - acc: 0.8868 - val_loss: 0.3451 - val_acc: 0.8803
Epoch 12/18
148/148 [==============================] - 3312s 22s/step - loss: 0.3164 - acc: 0.8914 - val_loss: 0.3437 - val_acc: 0.8855
Epoch 13/18
148/148 [==============================] - 3313s 22s/step - loss: 0.3180 - acc: 0.8902 - val_loss: 0.3428 - val_acc: 0.8838
Epoch 14/18
148/148 [==============================] - 3314s 22s/step - loss: 0.2941 - acc: 0.8938 - val_loss: 0.3244 - val_acc: 0.8940
Epoch 15/18
148/148 [==============================] - 3308s 22s/step - loss: 0.2770 - acc: 0.9035 - val_loss: 0.3232 - val_acc: 0.8855
Epoch 16/18
148/148 [==============================] - 3309s 22s/step - loss: 0.2372 - acc: 0.9175 - val_loss: 0.2991 - val_acc: 0.8923
Epoch 17/18
148/148 [===========================

100%|██████████| 794/794 [00:10<00:00, 73.42it/s]
/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [16]:
frame=pandas.DataFrame(invert[1]).transpose()
print(frame.mean())

0     0.000163
1     0.992824
2     0.000445
3     0.000085
4     0.000071
5     0.000485
6     0.000073
7     0.003916
8     0.000098
9     0.001170
10    0.000598
11    0.000073
dtype: float64


In [ ]:
"""Second attempt at ensemble. Feeds five InceptionResNetV2 models directly into an average, all in one model."""
size = (299,299,3)

image = Input(size)

model0 = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=image, input_shape=size)
x0 = model0.output
x0 = GlobalAveragePooling2D()(x0)
x0 = Dense(122, activation='tanh')(x0)
x0 = Dropout(.5)(x0)
predictions0 = Dense(12, activation='softmax')(x0)

model0 = Model(inputs=image, outputs=predictions0)

for layer in model0.layers[:618]:
   layer.trainable = False
for layer in model0.layers[618:]:
   layer.trainable = True

model0.load_weights('weights/model0.hdf5')

model1 = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=image, input_shape=size)
x1 = model1.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(122, activation='tanh')(x1)
x1 = Dropout(.5)(x1)
predictions1 = Dense(12, activation='softmax')(x1)

model1 = Model(inputs=image, outputs=predictions1)

for layer in model1.layers:
    layer.name='{}_1'.format(layer.name)

for layer in model1.layers[:618]:
    layer.trainable = False
for layer in model1.layers[618:]:
    layer.trainable = True

model1.load_weights('weights/model1.hdf5')

model2 = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=image, input_shape=size)
x2 = model2.output
x2 = GlobalAveragePooling2D()(x2)
x2 = Dense(122, activation='tanh')(x2)
x2 = Dropout(.5)(x2)
predictions2 = Dense(12, activation='softmax')(x2)

model2 = Model(inputs=image, outputs=predictions2)

for layer in model2.layers:
    layer.name='{}_2'.format(layer.name)

for layer in model2.layers[:618]:
   layer.trainable = False
for layer in model2.layers[618:]:
   layer.trainable = True

model2.load_weights('weights/model2.hdf5')

model3 = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=image, input_shape=size)
x3 = model3.output
x3 = GlobalAveragePooling2D()(x3)
x3 = Dense(122, activation='tanh')(x3)
x3 = Dropout(.5)(x3)
predictions3 = Dense(12, activation='softmax')(x3)

model3 = Model(inputs=image, outputs=predictions3)

for layer in model3.layers:
    layer.name='{}_3'.format(layer.name)

for layer in model3.layers[:618]:
   layer.trainable = False
for layer in model3.layers[618:]:
   layer.trainable = True

model3.load_weights('weights/model3.hdf5')

model4 = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=image, input_shape=size)
x4 = model4.output
x4 = GlobalAveragePooling2D()(x4)
x4 = Dense(122, activation='tanh')(x4)
x4 = Dropout(.5)(x4)
predictions4 = Dense(12, activation='softmax')(x4)

model4 = Model(inputs=image, outputs=predictions4)

for layer in model4.layers:
    layer.name='{}_4'.format(layer.name)

for layer in model4.layers[:618]:
   layer.trainable = False
for layer in model4.layers[618:]:
   layer.trainable = True

model4.load_weights('weights/model4.hdf5')

xe = Average()([model0.output, model1.output, model2.output, model3.output, model4.output])

ensemble = Model(inputs=image, outputs=xe)

ensemble.compile(optimizer=SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

Neither Ensemble model out performed the final ResNetInceptionV2 model, probably because the five models were too similar. There may also be better ways of combining them than average.